# AIFFELTHON Team GAL! Code Base
## Project \<<In The Name of Justice\>>

Team : 박수경, 이소명, 이태경, 정지영  

Goal : 대화문을 일상 대화문과 부적절한 상황의 대화문으로 분류한다. 부적절한 상황은 명시적인 언어폭력이 나타나는 상황뿐 아니라 맥락을 고려한 암묵적인 위협 상황까지 포함한다. 특히 암묵적인 언어폭력, 위협상황에 해당하는 대화문을 ‘협박', '갈취', '직장 내 괴롭힘', '기타 괴롭힘'의 세부 레이블로 분류하여 메신저, SNS, 커뮤니티 상에서 일어나는 부적절한 상황을 탐지하는 서비스에 활용할 수 있는 모델을 설계한다. 

## STEP 1. 작업환경 구성하기

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import keras as keras
from keras.models import load_model

import encoding_korbert as enc
import cleaning_and_normalizing as cleandnor

## STEP 2. 데이터 인코딩

In [ ]:
input_path = '<깃허브리포지토리의 path>' + "/input.txt" #local_host? git 서버?
input_df = enc.make_table(input_path)

In [ ]:
input_df = cleandnor.preprocess_input(input_df) #전처리

In [ ]:
input_x, input_y = enc.load_data(input_df) #입력할 데이터 준비 완료

## STEP 3. Ensembled model architecting

In [ ]:
# 모델 로드
model_1 = tf.keras.models.load_model('./saved/model_1.h5')
model_2 = tf.keras.models.load_model('./saved/model_2.h5')

# 새로운 모델 생성
def model_ens(x, threshold = (0.4, 0.6)):
    p1 = model_1(x)
    # threshold 범위 설정
    threshold_min = threshold[0]
    threshold_max = threshold[1]
    # p1이 threshold 범위 안에 있는지 확인

    if p1 >= threshold_min and p1 <= threshold_max: #imp
        
        p2 = model_2(x)
        #p2는 model_2가 가진 클래스 개수가 len 인 확률 벡터.
        
        # p3 = np.add(p2, 2) #p3 요소들의 범위는 2=<p3=<3 가 된다. (떨어뜨려놓기) 
        label = np.argmax(p2) + 2 
        
        return p1, np.max(p2), label # 결국 label 범위는 2, 3, 4..,n 이 됨
    
    else:

        label = np.argmax(p1)
        return p1, None, label # label은 0, 1


In [ ]:
def predict_class(x, dict:dict) -> str: #dict는 레이블 사전 입력하기.
    
    vector = model_ens(x)
    label = vector[2]
    predicted_class = dict[label]
    
    print('이 대화문은 ',predicted_class,'입니다.')


In [ ]:
class_dict = {0: '명시적인 언어폭력',\
                1: '일상적인 대화문',\
                2: '암묵적인 갈취 대화',\
                3: '암묵적인 기타 괴롭힘 대화',\
                4: '암묵적인 직장 내 괴롭힘 대화',\
                5: '암묵적인 협박 대화'}

## STEP 4. Show prediction over the web  
인코딩된 input data를 model_ens로 예측하여 그 결과를 웹을 통해 보여준다.  
(웹하고 join 하는 부분 해협!!) 

In [ ]:
predict_class(input_x,class_dict)